# Elastic Single Crystal Demo

In [1]:
import numpy as np

from polycrystal.elasticity.single_crystal import SingleCrystal
from polycrystal.utils.tensor_data.voigt_system import VoigtSystem

np.set_printoptions(precision=4)

## Example 1: Simple Material
In our first example, we use a simple and unrealistic material to show the effects of changing systems and units. 

There are three systems 
for representing the stiffness tensor as a 6 x 6 matrix: VOIGT_GAMMA, VOIGT_EPSILON and MANDEL. The first two systems use the Voigt method for representing symmetric 3 x 3 matrices as vectors with 6 components; see the other polycrystal demo (**working_with_tensors.ipynb**). The difference between the two methods is that the VOIGT_GAMMA matrix operates on the `gamma` shear strains, which are twice the `epsilon`  shear strains, but delivers the same stress in the standard Voigt system, as does the VOIGT_EPSILON matriux.  The Mandel system uses Mandel method for both the strains and stresses.

Units are specified by a string and are implemented using the python `pint` package. Most normal strings will work, e.g. "MPa", "GPa", "psi". 

Our first example is simply a stiffness matrix of all ones. This is not a realistic material, but it shows how the various systems are related. The `symmetry` and `cij` arguments are required. The `triclinic` symmetry, essentially no symmetry, can be used for any material; you simply provide the upper triangle (21 values) of the 6x6 stiffness matrix. The default system is VOIGT_GAMMA since it seems to be the most popular system in the literature. The default units are "GPa".

In [2]:
symm, cij = "triclinic", np.ones(21)
all_ones = SingleCrystal(symm, cij)
all_ones.system, all_ones.units, all_ones.stiffness

(<MatrixComponentSystem.VOIGT_GAMMA: 1>,
 'GPa',
 array([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]]))

Now we change the system to show how the stiffness matrix changes. Note how the "VOIGT_EPSILON" gives a non-symmetric matrix, and how the "MANDEL" gives a symmetric one. In fact, the "VOIGT_GAMMA" and "MANDEL" systems always give a symmetric matrix, and the "VOIGT_EPSILON" will produce a symmetric matrix only when the upper right 3x3 submatrix is all zero.

In [3]:
all_ones.system = "VOIGT_EPSILON"
all_ones.system, all_ones.units, all_ones.stiffness

(<MatrixComponentSystem.VOIGT_EPSILON: 2>,
 'GPa',
 array([[1., 1., 1., 2., 2., 2.],
        [1., 1., 1., 2., 2., 2.],
        [1., 1., 1., 2., 2., 2.],
        [1., 1., 1., 2., 2., 2.],
        [1., 1., 1., 2., 2., 2.],
        [1., 1., 1., 2., 2., 2.]]))

In [4]:
all_ones.system = "MANDEL"
all_ones.system, all_ones.units, all_ones.stiffness

(<MatrixComponentSystem.MANDEL: 3>,
 'GPa',
 array([[1.    , 1.    , 1.    , 1.4142, 1.4142, 1.4142],
        [1.    , 1.    , 1.    , 1.4142, 1.4142, 1.4142],
        [1.    , 1.    , 1.    , 1.4142, 1.4142, 1.4142],
        [1.4142, 1.4142, 1.4142, 2.    , 2.    , 2.    ],
        [1.4142, 1.4142, 1.4142, 2.    , 2.    , 2.    ],
        [1.4142, 1.4142, 1.4142, 2.    , 2.    , 2.    ]]))

Changing the units is simple and is independent of the system.

In [5]:
all_ones.units = "ksi"
all_ones.system, all_ones.units, all_ones.stiffness

(<MatrixComponentSystem.MANDEL: 3>,
 'ksi',
 array([[145.0377, 145.0377, 145.0377, 205.1143, 205.1143, 205.1143],
        [145.0377, 145.0377, 145.0377, 205.1143, 205.1143, 205.1143],
        [145.0377, 145.0377, 145.0377, 205.1143, 205.1143, 205.1143],
        [205.1143, 205.1143, 205.1143, 290.0755, 290.0755, 290.0755],
        [205.1143, 205.1143, 205.1143, 290.0755, 290.0755, 290.0755],
        [205.1143, 205.1143, 205.1143, 290.0755, 290.0755, 290.0755]]))

## Example 2: Computing Grain Stresses from Grain Strains
This example demonstrates more advanced capabilities of the elastic `SingleCrystal` class.  The information below is the entry in our demo materials database for LSHR, a nickel alloy, at 660 C.  We will use this data to show how to use the `apply_stiffness()` and `apply_compliance()` methods of the `SingleCrystal` class. 
```
- name     : lshr_660C
  units    : GPa
  symmetry : cubic
  system   : VOIGT_GAMMA
  moduli   :
    c11 : 175.4
    c12 : 107.8
    c44 : 106.2
  reference: >
           Boyce, D., P. Shade, W. Musinski, et al. 2020. “Estimation of Anisotropic
           Elastic Moduli from High Energy X-Ray Data and Finite Element Simulations.”
           Materialia 12 (August): 100795. https://doi.org/10.1016/j.mtla.2020.100795.
```  
The simplest way to instantiate this is to use the `materials_database` interface, but here we will instantiate it directly. See also the **materials-database.ipynb** demo.

In [6]:
symmetry = "cubic"
cij = (c11, c12, c44) = (175.4, 107.8, 106.2)
system = "VOIGT_GAMMA"
units = "GPa"
lshr = SingleCrystal(symmetry, cij, name="lshr_660C", system=system, units=units)

#  Show attributes:
lshr.system, lshr.units, lshr.stiffness

(<MatrixComponentSystem.VOIGT_GAMMA: 1>,
 'GPa',
 array([[175.4, 107.8, 107.8,   0. ,   0. ,   0. ],
        [107.8, 175.4, 107.8,   0. ,   0. ,   0. ],
        [107.8, 107.8, 175.4,   0. ,   0. ,   0. ],
        [  0. ,   0. ,   0. , 106.2,   0. ,   0. ],
        [  0. ,   0. ,   0. ,   0. , 106.2,   0. ],
        [  0. ,   0. ,   0. ,   0. ,   0. , 106.2]]))

Now that we have the elastic material, we shall demonstrate the `apply_stiffness()` method. Because the single crystal stiffness is relative to a crystallographic reference frame, you need to rotate the stiffness matrix if you want to apply it in another, rotated reference frame. A typical example would be *hexrd* output from grain fitting. The results include grain-averaged elastic strains in a fixed reference frame, usually the laboratory (or sample) frame, and the orientation matrices that convert crystallographic coordinates to the lab coordinates.  If you want to find the grain-averaged stresses, then you have convert the strains to the crystallographic frame, apply the stiffness matrix, then convert back to the lab frame. This is exactly what the `apply_stiffness()` method does. It takes an array of strain matrices (in the lab frame) `eps` and an array of orientation matrices `rmats`.  It returns the array of stress matrices (in the lab frame). Note that is independent of the underlying `system` for representing the stiffness matrix.  

### Step 1: Load the grain data
Normally, you might load the grain data from a *hexrd* `grains.out` file, but we didn't want to require the *hexrd* environment for this demo, so we just created a numpy file, `lshr7-100.npz`, with the grain-averaged strains and orientations. This data set is from the paper referenced above in materials database entry, on the esitmation of anisotropic moduli. The file contains the orientations and grain-averaged strains (as 6-component Voigt vectors) for 100 grains with 100% completeness. The strains are the difference between the measured strains at a loaded state (400 N) of an LSHR sample and the unloaded state.  

In [7]:
graindata = np.load("data/lshr-100.npz")
rmats, eps_s_voigt = graindata["rmats"], graindata["eps"]
print("average strain values\n", eps_s_voigt.mean(0))

average strain values
 [-7.3556e-04  2.4251e-03 -5.6491e-04 -3.5372e-05  8.3951e-06 -1.3186e-05]


### Step 2: Convert strain data to 3x3 matrices
Now we have the raw data. The `SingleCrystal.apply_stiffness()` and `SingleCrystal.apply_compliance()` methods act on arrays of 3x3 matrices
and return arrays of 3x3 matrices, so we need to convert the 6-vector strains to 3x3 matrices.

In [8]:
# Make matrices from Voigt-vector.
eps_s_mat = VoigtSystem.from_parts(symm=eps_s_voigt).matrices

### Step 3: Apply the stiffness matrices.  
Now that the strains are in 3x3 matrix form, we can use the `apply_stiffness()` method.

In [9]:
# Apply stiffness to get the stress matrices.
sig_s_matrices = lshr.apply_stiffness(eps_s_mat, rmats)

### Step 4: Convert the stresses back to the 6-component vectors.  
Here we, optionally, convert back to the Voigt 6-vector form and take the mean just to get an idea of the stress.

In [10]:
# Convert to the Voigt-vector form.
sig_s_voigt = VoigtSystem(sig_s_matrices).symm

print("average stress values\n", sig_s_voigt.mean(0))

average stress values
 [-0.0211  0.4695 -0.0086 -0.0012 -0.002  -0.0041]


### Step 5: Apply the Compliance to Recover Strains

In [11]:
eps_s_mats_cmp = lshr.apply_compliance(sig_s_matrices, rmats)
print("They are close:", np.allclose(eps_s_mats_cmp, eps_s_mat))

They are close: True
